In [1]:
import pandas as pd
import geopandas as gpd

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [8]:
## Malha de setores do BR
# https://www.ibge.gov.br/geociencias/downloads-geociencias.html?caminho=organizacao_do_territorio/malhas_territoriais/malhas_de_setores_censitarios__divisoes_intramunicipais/2021/Malha_de_setores_(shp)_Brasil

In [ ]:
## Download das tabelas e documentacao
# https://www.ibge.gov.br/estatisticas/sociais/populacao/9662-censo-demografico-2010.html?edicao=10410

# Selecao das variáveis

## Habitação einfraestrutura

* Porcentagem de domicílios do tipo cômodo
* Porcentagem de domicílios sem coleta de lixo
  * Domicilio01_UF.csv, V035
* Porcentagem de domicílios sem ligação à rede de abastecimento de água
  * Domicilio01_UF.csv, V106
* Porcentagem de domicílios sem banheiros ou sanitários
  * Domicilio01_UF.csv, V123
* Porcentagem de domicílios sem ligação à rede de esgoto ou fossa séptica
  * V017 adaptado
* Porcentagem de domicílios – outra forma de posse da moradia
  * N/A
* Número de banheiros por habitante
  * Adaptado V023

## Renda e escolaridade do responsável pelo domicílio

* Porcentagem de responsáveis por domicílio não alfabetizados
  * Responsavel02_UF.csv, adaptado V093
* Porcentagem de responsáveis por domicílio com menos de 30 anos não alfabetizados
* Porcentagem de responsáveis por domicílio com renda de até 3 salários mínimos
  * ResponsavelRenda_UF
* Porcentagem de responsáveis por domicílio com menos de 8 anos de estudo
* Anos médios de estudo do responsável pelo domicílio
* Renda média do responsável pelo domicílio

## Aspectos demográficos

* Número de domicílios particulares permanentes no setor censitário
* Número de domicílios improvisados no setor censitário
* Número de pessoas residentes no setor censitário
* Porcentagem de responsáveis por domicílios com menos de 30 anos
* Número médio de pessoas por domicílio



In [16]:
gdf_br = gpd.read_file('data/censo_2010/BR_Setores_2021.zip')

In [17]:
gdf_br.shape

(452248, 13)

In [7]:
gdf_br.NM_SIT.value_counts()

Área Urbana de Alta Densidade de Edificações     306095
Área Rural (exclusive aglomerados)                83569
Área Urbana de Baixa Densidade de Edificações     31869
Povoado                                           16832
Núcleo Urbano                                      7682
Lugarejo                                           5010
Massa de Água                                       894
Núcleo Rural                                        295
Name: NM_SIT, dtype: int64

In [5]:
gdf_br.columns

Index(['CD_SETOR', 'CD_SIT', 'NM_SIT', 'CD_UF', 'NM_UF', 'SIGLA_UF', 'CD_MUN',
       'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST', 'geometry'],
      dtype='object')

In [7]:
UFS = [
    'AC',
    'AL',
    'AM',
    'AP',
    'BA',
    'CE',
    'DF',
    'ES',
    'GO',
    'MA',
    'MG',
    'MS',
    'MT',
    'PA',
    'PB',
    'PE',
    'PI',
    'PR',
    'RJ',
    'RN',
    'RO',
    'RR',
    'RS',
    'SC',
    'SE',
    'SP1',
    'SP2',
    'TO'
]

In [8]:
## Hash para montar a base de dados
# {'file1': ['var1', 'var2', ..., 'var n'], ...,'file n': ['var1', 'var2', ..., 'var n']}

model_vars = {
    'Domicilio01': {
        'V035': 'seila1',
        'V106': 'seila2',
        'V123': 'seila3',
        'V023': 'seila4'
    },
    'Responsavel02': {
        'V093': 'seila5',
    },
}


In [14]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    dfs_uf = []
    for file in model_vars:
        file_path = f'data/censo_2010/csvs/{file}_{uf}.csv'
        fields = list(model_vars[file].keys())
        fields.append('Cod_setor')
        try:
            df = pd.read_csv(file_path, delimiter=';', usecols=fields).set_index('Cod_setor')
        except:
            df = pd.read_csv(file_path, delimiter=',', usecols=fields).set_index('Cod_setor')
        df.rename(columns=model_vars[file], inplace=True)
        dfs_uf.append(df)
        # break
    df = pd.concat(dfs_uf, axis=1)
    dfs.append(df)

Processando AC
Processando AL
Processando AM
Processando AP
Processando BA
Processando CE
Processando DF
Processando ES
Processando GO
Processando MA
Processando MG
Processando MS
Processando MT
Processando PA
Processando PB
Processando PE
Processando PI
Processando PR


/tmp/ipykernel_186212/2441012663.py:10: DtypeWarning: Columns (24,36,107,124) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=';', usecols=fields).set_index('Cod_setor')
/tmp/ipykernel_186212/2441012663.py:10: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter=';', usecols=fields).set_index('Cod_setor')


Processando RJ
Processando RN
Processando RO
Processando RR
Processando RS
Processando SC
Processando SE
Processando SP1
Processando SP2
Processando TO


In [15]:
pd.concat(dfs)

,seila4,seila1,seila2,seila3,seila5
Cod_setor,,,,,
120001305000001,0,279,134,11,242
120001305000002,3,331,223,19,293
120001305000003,0,473,253,102,404
120001305000004,2,49,16,8,42
120001305000005,1,0,0,40,71
...,...,...,...,...,...
172210705000014,0,3,0,14,25
172210705000015,37,0,0,64,68
172210705000016,56,0,0,71,50


In [27]:
dfs = []
for uf in UFS:
    print(f'Processando {uf}')
    file_path = f'data/censo_2010/csvs/Basico_{uf}.csv'
    # fields = list(model_vars[file].keys())
    # fields.append('Cod_setor')
    try:
        df = pd.read_csv(file_path, delimiter=';',  encoding='ISO-8859-1')
    except:
        df = pd.read_csv(file_path, delimiter=',', encoding='ISO-8859-1')
    # df.rename(columns=model_vars[file], inplace=True)
        # break
    df = pd.concat(dfs_uf, axis=1)
    dfs.append(df)

Processando AC


ValueError: No objects to concatenate

In [23]:
total = 0 
for df in dfs:
    total += df.shape[0]

In [24]:
total

310120